In [17]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
import tensorflow as tf

In [19]:
print(tf.__version__)

1.3.0


## Load data

In [20]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [21]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [22]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [23]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [77]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [25]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [26]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

## Prepare data

In [63]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train, random_state=42)

In [64]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [65]:
#X_train = np.transpose(X_train, (0, 2, 1))
#X_valid = np.transpose(X_valid, (0, 2, 1))
#har_X_test = np.transpose(har_X_test, (0, 2, 1))

In [66]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

# Build RNN model

## Hyperparameters

In [67]:
batch_size = 200
n_epochs = 200
learning_rate = 0.001

seq_len = 128
n_channels = 9
n_neurons = 100
n_layers = 3

# outputs
n_classes = 6

In [68]:
n_batches = X_train.shape[0] // batch_size

## CNN + RNN model

In [83]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [84]:
with tf.name_scope("cnn_rnn"):
    
    cnn = tf.layers.conv1d(X,
                           filters=32,
                           kernel_size=4,
                           strides=1,
                           padding="same",
                           activation=tf.nn.relu)
    print(cnn.shape)
    cnn = tf.reshape(cnn, (-1, 32, 128))
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, cnn, dtype=tf.float32)
    
    # only one RNN cell, the shape of states is (None, n_neurons)
    #states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states, n_classes)

(?, 128, 32)


In [85]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [86]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [87]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [88]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [89]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_first_cnn_rnn_model.ckpt")

Epoch:  0 Loss: 0.822897 Valid accuracy:  0.661045
Epoch:  10 Loss: 0.154491 Valid accuracy:  0.947225
Epoch:  20 Loss: 0.123808 Valid accuracy:  0.952666
Epoch:  30 Loss: 0.116499 Valid accuracy:  0.954298
Epoch:  40 Loss: 0.115157 Valid accuracy:  0.954842
Epoch:  50 Loss: 0.105123 Valid accuracy:  0.95593
Epoch:  60 Loss: 0.098916 Valid accuracy:  0.957018
Epoch:  70 Loss: 0.115285 Valid accuracy:  0.943961
Epoch:  80 Loss: 0.118064 Valid accuracy:  0.947769
Epoch:  90 Loss: 0.0992844 Valid accuracy:  0.95593
Epoch:  100 Loss: 0.0915025 Valid accuracy:  0.957563
Epoch:  110 Loss: 0.090007 Valid accuracy:  0.957018
Epoch:  120 Loss: 0.0992666 Valid accuracy:  0.965724
Epoch:  130 Loss: 0.0819517 Valid accuracy:  0.962459
Epoch:  140 Loss: 0.079552 Valid accuracy:  0.966268
Epoch:  150 Loss: 0.138266 Valid accuracy:  0.953754
Epoch:  160 Loss: 0.0836969 Valid accuracy:  0.963003
Epoch:  170 Loss: 0.106737 Valid accuracy:  0.945049
Epoch:  180 Loss: 0.0856682 Valid accuracy:  0.962459


In [90]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_first_cnn_rnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_first_cnn_rnn_model.ckpt
valid Accuracy:  0.960283
Test Accuracy:  0.911775


## Basic RNN model with dropout

In [110]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [111]:
with tf.name_scope("cnn_lstm"):

    cnn = tf.layers.conv1d(X,
                           filters=32,
                           kernel_size=4,
                           strides=1,
                           padding="same",
                           activation=tf.nn.relu)
    
    cnn = tf.reshape(cnn, (-1, 32, 128))    
    
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    outputs, states = tf.nn.dynamic_rnn(cell, cnn, dtype=tf.float32)
    
    # only one RNN cell, the shape of states is (None, n_neurons)
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states_concat, n_classes)

(?, 128, 32)


In [112]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [113]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [114]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [115]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [116]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_lstm_model.ckpt")

Epoch:  0 Loss: 0.586714 Valid accuracy:  0.746464
Epoch:  10 Loss: 0.132703 Valid accuracy:  0.939608
Epoch:  20 Loss: 0.117247 Valid accuracy:  0.94124
Epoch:  30 Loss: 0.10133 Valid accuracy:  0.957563
Epoch:  40 Loss: 0.127094 Valid accuracy:  0.948857
Epoch:  50 Loss: 0.104333 Valid accuracy:  0.949946
Epoch:  60 Loss: 0.134786 Valid accuracy:  0.935256
Epoch:  70 Loss: 0.0816167 Valid accuracy:  0.966268
Epoch:  80 Loss: 0.0848946 Valid accuracy:  0.965724
Epoch:  90 Loss: 0.0891107 Valid accuracy:  0.96518
Epoch:  100 Loss: 0.0993783 Valid accuracy:  0.961915
Epoch:  110 Loss: 0.0623821 Valid accuracy:  0.974973
Epoch:  120 Loss: 0.0606186 Valid accuracy:  0.976605
Epoch:  130 Loss: 0.0568671 Valid accuracy:  0.983134
Epoch:  140 Loss: 0.0655618 Valid accuracy:  0.978237
Epoch:  150 Loss: 0.0692112 Valid accuracy:  0.980413
Epoch:  160 Loss: 0.0727688 Valid accuracy:  0.980958
Epoch:  170 Loss: 0.0642399 Valid accuracy:  0.981502
Epoch:  180 Loss: 0.103562 Valid accuracy:  0.957

In [117]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_lstm_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_dropout_model.ckpt
valid Accuracy:  0.974429
Test Accuracy:  0.91415


# CNN + Three LSTM layers with  early stopping

In [118]:
# maximum number of epoch
n_epochs=600

In [123]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [124]:
with tf.name_scope("cnn_lstm"):
    
    cnn_layer = tf.layers.conv1d(X,
                                 filters=32,
                                 kernel_size=2,
                                 strides=1,
                                 padding="same",
                                 activation=tf.nn.relu)
    cnn_layer = tf.reshape(cnn_layer, (-1, 32, 128))
    
    cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
                 for _ in range(n_layers)]
    
    multi_cells = tf.contrib.rnn.MultiRNNCell(cells)
   
    outputs, states = tf.nn.dynamic_rnn(multi_cells, cnn_layer, dtype=tf.float32)
    #top_layer_h_state = states[-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states_concat, n_classes)

In [126]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [127]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [128]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [129]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [130]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=150

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_cnn_lstm_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  0.632894 Valid accuracy:  0.708379
Epoch:  5 Loss:  0.155949 Valid accuracy:  0.939608
Epoch:  10 Loss:  0.138249 Valid accuracy:  0.944505
Epoch:  15 Loss:  0.157683 Valid accuracy:  0.931447
Epoch:  20 Loss:  0.107463 Valid accuracy:  0.95593
Epoch:  25 Loss:  0.110758 Valid accuracy:  0.947769
Epoch:  30 Loss:  0.0983662 Valid accuracy:  0.957563
Epoch:  35 Loss:  0.106005 Valid accuracy:  0.952122
Epoch:  40 Loss:  0.0941343 Valid accuracy:  0.956474
Epoch:  45 Loss:  0.101297 Valid accuracy:  0.956474
Epoch:  50 Loss:  0.0918952 Valid accuracy:  0.956474
Epoch:  55 Loss:  0.101611 Valid accuracy:  0.956474
Epoch:  60 Loss:  0.238746 Valid accuracy:  0.925462
Epoch:  65 Loss:  0.0934247 Valid accuracy:  0.959195
Epoch:  70 Loss:  0.102751 Valid accuracy:  0.951034
Epoch:  75 Loss:  0.0840065 Valid accuracy:  0.955386
Epoch:  80 Loss:  0.0896026 Valid accuracy:  0.957563
Epoch:  85 Loss:  0.0866468 Valid accuracy:  0.959739
Epoch:  90 Loss:  0.100553 Valid accuracy:

In [131]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_lstm_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_lstm_early_stopping_model.ckpt
valid Accuracy:  0.980958
Test Accuracy:  0.902952


## CNN with Bi-directional LSTM

In [148]:
n_epochs=200

In [151]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [152]:
with tf.name_scope("rnn"):
    

    cnn_layer = tf.layers.conv1d(X,
                                 filters=32,
                                 kernel_size=2,
                                 strides=1,
                                 padding="same",
                                 activation=tf.nn.relu)
    cnn_layer = tf.transpose(cnn_layer, perm=(0, 2, 1))
    
    fw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    
    bw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
        
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, cnn_layer, dtype=tf.float32)
    #print(states.shape)
    #top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states[-1][-1])
    logits = tf.layers.dense(states_concat, n_classes)

In [153]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [154]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [155]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [156]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [157]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
             
        if epoch % 10 == 0:
            
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_bilstm_model.ckpt")

Epoch:  0 Loss: 0.65961 Valid accuracy:  0.696953
Epoch:  10 Loss: 0.172222 Valid accuracy:  0.925462
Epoch:  20 Loss: 0.140146 Valid accuracy:  0.945049
Epoch:  30 Loss: 0.106622 Valid accuracy:  0.947225
Epoch:  40 Loss: 0.127846 Valid accuracy:  0.952122
Epoch:  50 Loss: 0.11166 Valid accuracy:  0.958651
Epoch:  60 Loss: 0.117796 Valid accuracy:  0.95593
Epoch:  70 Loss: 0.110877 Valid accuracy:  0.964635
Epoch:  80 Loss: 0.175128 Valid accuracy:  0.955386
Epoch:  90 Loss: 0.150007 Valid accuracy:  0.963547
Epoch:  100 Loss: 0.180045 Valid accuracy:  0.959739
Epoch:  110 Loss: 0.106107 Valid accuracy:  0.973885
Epoch:  120 Loss: 0.10993 Valid accuracy:  0.963003
Epoch:  130 Loss: 0.16914 Valid accuracy:  0.964091
Epoch:  140 Loss: 0.215812 Valid accuracy:  0.963003
Epoch:  150 Loss: 0.164962 Valid accuracy:  0.963547
Epoch:  160 Loss: 0.164917 Valid accuracy:  0.967356
Epoch:  170 Loss: 0.135381 Valid accuracy:  0.972796
Epoch:  180 Loss: 0.187119 Valid accuracy:  0.9679
Epoch:  190

In [158]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_bilstm_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_bilstm_model.ckpt
valid Accuracy:  0.976605
Test Accuracy:  0.922973


# Bi-directional Basic RNN with dropout and early stopping

In [159]:
n_epochs=600

In [164]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [165]:
with tf.name_scope("cnn_lstm"):
    
    cnn_layer = tf.layers.conv1d(X,
                                filters=32,
                                kernel_size=2,
                                strides=1,
                                padding="same",
                                activation=tf.nn.relu)
    cnn_layer = tf.transpose(cnn_layer, perm=(0, 2, 1))
    
    fw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    bw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
    
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, cnn_layer, dtype=tf.float32)
    
    states_concat = tf.concat(axis=1, values=states[-1][-1])
    logits = tf.layers.dense(states_concat, n_classes)

In [166]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [167]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [168]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [169]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [170]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=100

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_cnn_bilstm_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  0.911722 Valid accuracy:  0.589772
Epoch:  5 Loss:  0.282099 Valid accuracy:  0.885745
Epoch:  10 Loss:  0.212058 Valid accuracy:  0.906964
Epoch:  15 Loss:  0.18817 Valid accuracy:  0.928727
Epoch:  20 Loss:  0.1688 Valid accuracy:  0.937432
Epoch:  25 Loss:  0.144179 Valid accuracy:  0.94124
Epoch:  30 Loss:  0.145876 Valid accuracy:  0.945049
Epoch:  35 Loss:  0.126862 Valid accuracy:  0.949401
Epoch:  40 Loss:  0.108876 Valid accuracy:  0.954298
Epoch:  45 Loss:  0.122585 Valid accuracy:  0.949946
Epoch:  50 Loss:  0.121778 Valid accuracy:  0.954842
Epoch:  55 Loss:  0.100945 Valid accuracy:  0.955386
Epoch:  60 Loss:  0.130247 Valid accuracy:  0.954842
Epoch:  65 Loss:  0.113569 Valid accuracy:  0.95321
Epoch:  70 Loss:  0.11305 Valid accuracy:  0.952666
Epoch:  75 Loss:  0.124593 Valid accuracy:  0.949946
Epoch:  80 Loss:  0.133599 Valid accuracy:  0.954298
Epoch:  85 Loss:  0.170425 Valid accuracy:  0.947769
Epoch:  90 Loss:  0.154386 Valid accuracy:  0.948857
E

In [62]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_bilstm_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_early_stopping_model.ckpt
valid Accuracy:  0.970076
Test Accuracy:  0.904649
